In [1]:
import pandas as pd
import numpy as np
from glob import glob


In [12]:
store = "VR_B"
file_name = store+"*.csv"
save_to_name = store+"_to_VO_4_20.txt"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
df[["Barcode","Quantity"]].to_csv(save_to_name,sep="\t",header=None,index=False)
f = glob(file_name)
f.sort()
print(f)
print(df.Quantity.sum())
print(len(df_list))

['VR_B_1419.csv']
14
1


In [3]:
file_name = "in_transit.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)

df = df.rename({"Trnf#":"TransferNumber"},axis=1)
df.groupby(["TransferNumber","To"]).agg({"O Qty":"sum"})

in_transit.csv


,,O Qty
TransferNumber,To,
2849,A1,76
2850,A1,67
2851,A1,67
2852,A1,72
2854,A1,1495
...,...,...
3260,A1,64
3261,A1,51
3262,A1,76


In [2]:
file_name = "7_pallet.csv"
df2 = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df2 = pd.concat(df_list,axis=0)

df2 = df2.rename({"Trnf#":"TransferNumber"},axis=1)
df2

ValueError: No objects to concatenate

In [81]:
df2

,TransferNumber,Priority,Sts,Prt,From,To,Cost,Via,Reference,Ent Date,Shp Date,Rec Date,ETA Date,Order,Style,O Qty,shp Qty,Rec Qty
0,3065,NaN,T,NaN,VS,A1,FOB,F11,4.1.21 CW,4/1/2021,4/19/2021,6/14/2021,4/22/2021,NaN,V126,1,1,0
1,3065,NaN,T,NaN,VS,A1,FOB,F11,4.1.21 CW,4/1/2021,4/19/2021,6/14/2021,4/22/2021,NaN,V150,1,1,0
2,3065,NaN,T,NaN,VS,A1,FOB,F11,4.1.21 CW,4/1/2021,4/19/2021,6/14/2021,4/22/2021,NaN,V237,1,1,0
3,3065,NaN,T,NaN,VS,A1,FOB,F11,4.1.21 CW,4/1/2021,4/19/2021,6/14/2021,4/22/2021,NaN,V302,3,3,0
4,3065,NaN,T,NaN,VS,A1,FOB,F11,4.1.21 CW,4/1/2021,4/19/2021,6/14/2021,4/22/2021,NaN,V314,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,3241,NaN,T,NaN,VS,A1,FOB,F11,NaN,4/9/2021,4/19/2021,6/14/2021,4/22/2021,NaN,VW416,1,1,0
4163,3241,NaN,T,NaN,VS,A1,FOB,F11,NaN,4/9/2021,4/19/2021,6/14/2021,4/22/2021,NaN,VW418,1,1,0
4164,3241,NaN,T,NaN,VS,A1,FOB,F11,NaN,4/9/2021,4/19/2021,6/14/2021,4/22/2021,NaN,VW425,3,3,0
4165,3241,NaN,T,NaN,VS,A1,FOB,F11,NaN,4/9/2021,4/19/2021,6/14/2021,4/22/2021,NaN,VW426,1,1,0


In [53]:
df4 = df2.groupby("TransferNumber").agg({"O Qty":sum})
df5 = df4.loc[df4["O Qty"] <=150]
fc_list = df5.index

In [79]:
df2.loc[df2.TransferNumber==3243]

,TransferNumber,Priority,Sts,Prt,From,To,Cost,Via,Reference,Ent Date,Shp Date,Rec Date,ETA Date,Order,Style,O Qty,shp Qty,Rec Qty


In [72]:
fc_list_str = np.char.mod('%d', fc_list)

In [74]:
for i in sheet_arry:
    if i not in fc_list_str:
        print(i)
        

3066
3243
3242
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253


In [ ]:
#3066,

In [75]:
for i in fc_list_str:
    if i not in sheet_arry:
        print(i)
        

In [73]:
spreadsheet_df = pd.read_csv("spreadsheet.csv").dropna()
spreadsheet_df.TO = spreadsheet_df.TO.apply(lambda x:x.split("T")[1].replace("B","").replace("-",""))
sheet_arry = spreadsheet_df.TO.unique()

In [3]:
file_name = "transfer_receiving.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
df = df.loc[df.From.isin(["A1","NR"])]

df = df.loc[~df.To.isin(["A1","NR"])]
df = df.rename({"Trnf#":"TransferNumber"},axis=1)
df.groupby(["TransferNumber","To"]).agg({"O Qty":"sum"})

transfer_receiving.csv


,,O Qty
TransferNumber,To,
2844,V3,69
3410,V3,761
3411,VR,993
3412,V5,1063
3413,V4,812
3414,VM,987
3415,V6,672
3425,VR,2652
3426,V5,1775


In [4]:
order_num = 3333
file_name = str(order_num) + ".csv"

In [5]:
def change_sizes(string):
    if string == "S":
        return "SML"
    if string == "XS":
        return "XSM"
    if string == "XXS":
        return "XXS"
    if string == "M":
        return "MED"
    if string == "L":
        return "LRG"
    if string == "XL":
        return "XLG"
    if string == "XXL":
        return "XXL"
    else:
        return string
def apply_numeric_size(string):
    if string.isnumeric():
        string = string.zfill(3)
    return string
df["Size"] = df.Size.apply(apply_numeric_size)
df["Size"] = df.Size.apply(change_sizes)
df["SKU_full"] = df["Style"] + df["Clr"] + df["Size"]

In [6]:
purchase_order = pd.read_csv(file_name)
purchase_order.groupby("Barcode").agg({"Qty Ordered":"sum"})
df_2821= df.loc[df["TransferNumber"] == order_num]
df_2821["UPC Code"] = df_2821["UPC Code"].astype("int").astype("str")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
check_0 = purchase_order.groupby("SKU").agg({"Qty Ordered":"sum"})["Qty Ordered"] - df_2821.groupby("SKU_full").agg({"Shp Qty":"sum"})["Shp Qty"]
check_0.loc[check_0 != 0].index

Index(['VW164HTWMED'], dtype='object', name='SKU')

In [8]:
for item_id in check_0.loc[check_0 != 0].index:
    print(item_id)
    print("Received")
    print(purchase_order.loc[purchase_order.SKU == item_id]["Qty Ordered"])
    print("On FULL circle")
    print(df_2821.loc[df_2821.SKU_full == item_id]["Shp Qty"])
    print("=================")


VW164HTWMED
Received
144    1
Name: Qty Ordered, dtype: int64
On FULL circle
43207    2
Name: Shp Qty, dtype: int64


In [17]:
item_id = "VW339HTGSML"
print("Received")
print(purchase_order.loc[purchase_order.SKU == item_id]["Qty Ordered"])
print("On FULL circle")
df_2821.loc[df_2821.SKU_full == item_id]["Shp Qty"]


Received
Series([], Name: Qty Ordered, dtype: int64)
On FULL circle


Series([], Name: Shp Qty, dtype: int64)

In [8]:
import pandas as pd
df = pd.read_csv("transfer_VO.csv")
df.columns

Index(['Trnf#', 'Line', 'Priority', 'Sts', 'Style', 'Clr', 'Style Description',
       'Color Description', 'Size Type', 'Size', 'O Qty', 'Shp Qty', 'Rec Qty',
       'Style Color Std Cost', 'Trns Value', 'Markdown MSRP', 'Outlet MSRP',
       'Wholesale Price', 'Segment Code', 'Inventory Classification', 'Sts.1',
       'Prt', 'From', 'To', 'Cost', 'Via', 'Reference', 'Ent Date', 'Shp Date',
       'Rec Date', 'ETA Date', 'Order', 'Fabric Content', 'HTS Code',
       'Hts Description', 'Country Of Origin', 'COO Description',
       'Style Color FOB', 'BOL #', 'Division', 'Division Description', 'Label',
       'Label Description', 'Category', 'Category Description',
       'NAFTA Compliant', 'UPC Code'],
      dtype='object')

In [21]:
to_number = 2804
to_numbers = [2804,2805,2806,2802,2803]
df = pd.read_csv("transfer_VO.csv",dtype={"UPC Code":"str"})
df = df.rename({"Trnf#":"transfer_num"},axis=1)
for to_number in to_numbers:
    
    
    small_df = df.loc[df.transfer_num == to_number][["UPC Code","Rec Qty"]]
    small_df.to_csv(str(to_number)+".txt",sep="\t",header=None,index=False)
    display(small_df)

,UPC Code,Rec Qty
0,843084176946,1
1,843084137459,1
2,843084137558,1
3,843084126613,1
4,843084126668,1
...,...,...
119,840159737446,3
120,843084172450,1
121,843084162772,1
122,843084149179,1


,UPC Code,Rec Qty
124,843084165438,1
125,843084166855,5
126,843084166862,4
127,843084166879,4
128,843084101795,1
129,843084167203,1
130,843084142064,1
131,843084142163,1
132,843084182831,1
133,843084153367,1


,UPC Code,Rec Qty
161,843084137947,1
162,843084138210,1
163,843084106257,1
164,867810000930,2
165,817718025801,1
...,...,...
245,843084150946,1
246,843084151356,1
247,843084154746,1
248,843084154760,1


,UPC Code,Rec Qty


,UPC Code,Rec Qty
